In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 

In [39]:
rating =pd.read_csv("C:/Users/Dareen/OneDrive/Desktop/recomindation/movie/ratings.csv") 
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [40]:
movie = pd.read_csv("C:/Users/Dareen/OneDrive/Desktop/recomindation/movie/movies.csv")
movie = movie[['title', 'genres','movieId']]
movie.head()

,title,genres,movieId
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
1,Jumanji (1995),Adventure|Children|Fantasy,2
2,Grumpier Old Men (1995),Comedy|Romance,3
3,Waiting to Exhale (1995),Comedy|Drama|Romance,4
4,Father of the Bride Part II (1995),Comedy,5


In [41]:
movie_data = rating.merge(movie, on='movieId')
movie_data.head()
sample  =movie_data.head(800000)
sample

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,851527569,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,849082742,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,835562174,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,846509384,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...,...
799995,64188,1089,4.0,1064196856,Reservoir Dogs (1992),Crime|Mystery|Thriller
799996,64191,1089,5.0,1128759098,Reservoir Dogs (1992),Crime|Mystery|Thriller
799997,64194,1089,5.0,891585125,Reservoir Dogs (1992),Crime|Mystery|Thriller
799998,64202,1089,2.0,1159195885,Reservoir Dogs (1992),Crime|Mystery|Thriller


In [42]:

user_item_matrix = sample.pivot(index=['userId'], columns=['movieId'], values='rating').fillna(0)
user_item_matrix

movieId,2,29,32,47,50,112,151,223,253,260,...,589,593,653,919,924,1009,1036,1079,1080,1089
userId,,,,,,,,,,,,,,,,,,,,,
1,3.5,3.5,3.5,3.5,3.5,3.5,4.0,4.0,4.0,4.0,...,3.5,3.5,3.0,3.5,3.5,3.5,4.0,4.0,3.5,3.5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,4.0,0.0,5.0,0.0,0.0,5.0,0.0,5.0,...,4.0,5.0,0.0,4.0,5.0,0.0,0.0,4.0,0.0,5.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,5.0,3.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138488,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138489,0.0,4.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
from sklearn.neighbors import NearestNeighbors
cf_knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
cf_knn_model.fit(user_item_matrix)


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [44]:
from fuzzywuzzy import process

In [45]:
movie_names = pd.DataFrame({
    'title': movie['title'],
    'movieId': movie['movieId']
})

In [51]:
from sklearn.metrics.pairwise import cosine_similarity
movie_similarity = cosine_similarity(user_item_matrix.T)


In [59]:
import pandas as pd
from fuzzywuzzy import process

def movie_recommender_engine(movie_name, matrix, movie_similarity, n_recs):
    # Check if the movie name exists in the movie_names DataFrame
    if movie_name not in movie_names['title'].values:
        raise ValueError(f"The movie '{movie_name}' is not in the database.")
    
    # Extract the movie ID for the given movie name
    movie_id = process.extractOne(movie_name, movie_names['title'])
    if movie_id is None:
        raise ValueError(f"No matching movie found for '{movie_name}'")
    
    movie_id = movie_id[2]
    
    # Ensure movie_id is an integer and within range
    if not isinstance(movie_id, int) or movie_id >= len(movie_similarity):
        raise IndexError(f"Movie ID {movie_id} is out of bounds for the similarity matrix.")
    
    # Get similarity scores for the movie
    sim_scores = list(enumerate(movie_similarity[movie_id]))
    
    # Sort movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the top N most similar movies
    sim_scores = sim_scores[1:n_recs+1]  # Exclude the movie itself (self-similarity)
    
    # Movie indices and scores
    movie_indices = [i[0] for i in sim_scores]
    movie_scores = [i[1] for i in sim_scores]
    
    # Get movie titles
    recommendations = [{'Title': movie_names['title'][i], 'Similarity': score} for i, score in zip(movie_indices, movie_scores)]
    
    return pd.DataFrame(recommendations, index=range(1, n_recs+1))

# Example usage
try:
    n_recs = 10
    recommendations = movie_recommender_engine('Jumanji (1995)', user_item_matrix, movie_similarity, n_recs)
    print(recommendations)
except Exception as e:
    print(f"An error occurred: {e}")


                                 Title  Similarity
1                        Casino (1995)    0.338415
2              Grumpier Old Men (1995)    0.328645
3                    Get Shorty (1995)    0.296813
4                     GoldenEye (1995)    0.264130
5             Leaving Las Vegas (1995)    0.263584
6       American President, The (1995)    0.259971
7   Father of the Bride Part II (1995)    0.254247
8   Dracula: Dead and Loving It (1995)    0.251153
9                  Tom and Huck (1995)    0.243859
10                       Powder (1995)    0.237104
